In [616]:
import pandas as pd
import numpy as np
from math import floor

In [617]:
#data_raw = pd.read_csv("resources/apple_stock.csv")
data_raw = pd.read_csv("resources/tesla.csv")
#data_raw = pd.read_csv("microsoft_stock.csv")
#data_raw = pd.read_csv("resources/kodak_stock.csv")
#data_raw = pd.read_csv("meta_stock.csv")

In [618]:
data_raw=data_raw.reindex(index=data_raw.index[::-1])

data_raw

,Date,Close/Last,Volume,Open,High,Low
1258,02/08/2017,$52.416,19562140,$51.47,$52.672,$51.24
1257,02/09/2017,$53.84,39063000,$53.25,$54.236,$53.23
1256,02/10/2017,$53.846,18091680,$53.958,$54.19,$53.222
1255,02/13/2017,$56.12,35115360,$54.148,$56.158,$54.102
1254,02/14/2017,$56.196,36707250,$55.806,$57.478,$55.722
...,...,...,...,...,...,...
4,02/01/2022,$931.25,24379450,$935.21,$943.7,$905
3,02/02/2022,$905.66,22264350,$928.18,$931.5,$889.41
2,02/03/2022,$891.14,26285190,$882,$937,$880.52
1,02/04/2022,$923.32,24541820,$897.22,$936.5,$881.17


In [619]:
data_raw[["Close/Last"]]=data_raw[["Close/Last"]].astype('str').replace({'\$':' '}, regex=True).astype('float')
data_raw[["Open"]]=data_raw[["Open"]].astype('str').replace({'\$':' '}, regex=True).astype('float')
data_raw[["High"]]=data_raw[["High"]].astype('str').replace({'\$':' '}, regex=True).astype('float')
data_raw[["Low"]]=data_raw[["Low"]].astype('str').replace({'\$':' '}, regex=True).astype('float')

data_preprocessed=data_raw.copy()

In [620]:
def stochastic_oscillator(close,low,high):
    return (close-low)/(high-low) * 100

In [621]:
data_preprocessed["Highest14"] = data_preprocessed["High"].rolling(window=14).max()
data_preprocessed["Lowest14"] = data_preprocessed["Low"].rolling(window=14).min()
data_preprocessed=data_preprocessed[data_preprocessed["Highest14"]!=-1]
data_preprocessed=data_preprocessed[~data_preprocessed["Highest14"].isna()]
data_preprocessed["StochasticOscillator14d"] = data_preprocessed.apply(lambda row:stochastic_oscillator(row['Close/Last'],row['Lowest14'],row['Highest14']), axis=1)
data_preprocessed["StochasticOscilaltorDiff"] = data_preprocessed["StochasticOscillator14d"].diff()


In [622]:
df=data_preprocessed
print(df)

            Date  Close/Last    Volume     Open     High       Low  Highest14  \
1245  02/28/2017      49.998  30369450   48.838   50.200   48.7800    57.4780   
1244  03/01/2017      50.004  24024815   50.836   50.970   49.8220    57.4780   
1243  03/02/2017      50.096  16728755   49.942   50.656   49.6540    57.4780   
1242  03/03/2017      50.314  14627405   50.148   50.380   49.8000    57.4780   
1241  03/06/2017      50.242  16768005   49.582   50.340   49.5020    57.4780   
...          ...         ...       ...      ...      ...       ...        ...   
4     02/01/2022     931.250  24379450  935.210  943.700  905.0000  1115.6000   
3     02/02/2022     905.660  22264350  928.180  931.500  889.4100  1115.6000   
2     02/03/2022     891.140  26285190  882.000  937.000  880.5200  1070.7899   
1     02/04/2022     923.320  24541820  897.220  936.500  881.1700  1070.7899   
0     02/07/2022     907.340  20331490  923.790  947.770  902.7089  1054.6699   

      Lowest14  StochasticO

## Testing

In [623]:
money= 1000.0

### Buy & Hold

In [624]:
if_bought_start = money/df[["Close/Last"]].iloc[0]*df[["Close/Last"]].iloc[len(df)-1]
if_bought_start[0]

18147.525901036042

### Using Stochastic oscillator 14days, buy on 20, sell on 80

In [626]:
data=[]
for i,row in df.iterrows():
    if i==0:
        continue;
    data.append([row[['Date']][0],money,row[["StochasticOscillator14d"]][0],row[["Close/Last"]][0],stocks])
    if i==0:
        money= 1000
        stocks = 0
    elif (row["StochasticOscillator14d"]<20) & (money>2*row[["Close/Last"]][0]) & (row['StochasticOscilaltorDiff']>0.5):
        stocks = stocks+floor(money/row[["Close/Last"]][0])
        money = money-stocks*row[["Close/Last"]][0]
    elif (row["StochasticOscillator14d"]>80) & (stocks>0) & (row['StochasticOscilaltorDiff']<0):
        money = money+stocks*row[["Close/Last"]][0]
        stocks=0
        
print(stocks)
print(money)

4
211.58199999999988


In [627]:
df=pd.DataFrame(data,columns=["Date","money","StochasticOscillator14d","Close","Stocks"])

In [628]:
df.to_excel("output.xlsx")